<a href="https://colab.research.google.com/github/Buzzrightear/CO885_dissertation/blob/master/RandomForrest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Taken from https://colab.research.google.com/drive/1WJjty6Q87WZeqBZVy6--cgrnuW4p50PP#scrollTo=1iEXEPxRNNUX 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [3]:
df = pd.read_csv('/root/wind_data_1.csv', parse_dates=['Date time',]) #Read in csv to dataframe
df=df[['Date time', 'Power avg']].dropna() # reduce dataframe to just 2 columns and drop any blank rows

print(df.head())


FileNotFoundError: ignored

**train_forest** takes a data frame, splits it into training and validation sets (based on a given number of records for the validation set), and fits a random forest.

**check_predictions** sets up a 2x2 grid for subplots, and calls predictions_plot for all training and validation results, plus the last week of the training set and the first week of the validation set. The theory is that the Random Forest's accuracy will degrade as the electrical system changes (e.g. additional rooftop solar capacity changes the relationship of sunny days) - but it should be more accurate in the short term.

**predictions_plot** calculates mean absolute error and maximum absolute error, then plots the results

In [ ]:

def train_forest(df, split_point):
  train_features = df.drop(['Power avg'], axis = 1).head(len(df)-split_point)   #Get rid of the 'Power avg' columns inc. column heading and keep all rows up to split_point
  train_labels = df['Power avg'].head(len(df)-split_point)                      #Just return 'Power avg' column without col heading up to split-point
  val_features = df.drop(['Power avg'], axis = 1).tail(split_point)             #Get rid of the 'Power avg' columns inc. column heading and keep all rows after split_point
  val_labels = df['Power avg'].tail(split_point)                                #Just return 'Power avg' column without col heading after split-point

  rf = RandomForestRegressor(n_estimators = 50, random_state = 42)
  rf.fit(train_features, train_labels)  #Train RF on training features (dates) and target classifcations / labels (power avg values)

  return [rf, train_features, train_labels, val_features, val_labels]

def check_predictions(rf, train_features, train_labels, val_features, val_labels):
  fig,axes  = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
  fig.tight_layout(pad=6.0)
  predictions_plot(rf, train_features, train_labels, axes[0,0])
  predictions_plot(rf, val_features, val_labels, axes[0,1])
  predictions_plot(rf, train_features.tail(7*48), train_labels.tail(7*48), axes[1,0])
  predictions_plot(rf, val_features.head(7*48), val_labels.head(7*48), axes[1,1])

def predictions_plot(rf, features, labels, ax):
  predictions = rf.predict(features) # Each call to this function just gives a set of features (dates) as a param and returns a dataframe of 'labels' (power avg values)
  print("Predictions head " + predictions.head())
  errors = abs(predictions - labels)
  print("Errors head " + errors.head())
  
  title='MAE:' + str(round(np.mean(errors), 2)) + \
        ' MAX_AE:' + str(round(np.max(abs(errors)), 2)) + \
        ' RMSE:' + str(round(np.sqrt(np.mean(np.square(errors))), 2))
  pd.DataFrame({'labels': labels, 
                'predictions': predictions, 
                'Date time': features['Date time']}).plot(title=title, ax=ax, x='Date time')
                
[rf, train_features, train_labels, val_features, val_labels] = train_forest(df, 365*144) #The last year's worth of entries are reserved as a validation set (144 entries per day * 365 days)
check_predictions(rf, train_features, train_labels, val_features, val_labels) 

In [ ]:
"""
abs_errors = abs(rf.predict(val_features.drop(['Power avg'], axis = 1)) - val_labels)
worst_index = abs_errors.tolist().index(max(abs_errors))
fig,axes  = plt.subplots(nrows=1, ncols=1, figsize=(6,6))
predictions_plot(rf, val_features.head(worst_index + 3*48).tail(7*48), val_labels.head(worst_index + 3*48).tail(7*48), axes)
"""